<a href="https://colab.research.google.com/github/PHaasbroek/Google-Colab/blob/main/Fluid-Solver.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The aim of this notebook is to simulate a simple numerical domain.

1. Create a laminar, steady, fluid solver for one dimensional pipe flow.
2. Create a RANS turbulence model.
3. Employ the turbulence model in the one dimensional pipe flow.
4. Employ the turbulence model for flow past a backward facing step.
5. Employ the turbulence model for flow past an aerofoil.

The incompressible Navier-Stokes (NS) equations for three dimensions are given as:

$$\frac{\partial u}{\partial t} + u \frac{\partial u}{\partial x} + v \frac{\partial u}{\partial y} + w \frac{\partial u}{\partial z} = -\frac{1}{\rho} \frac{\partial p}{\partial x} + \nu \left(\frac{\partial^2 u}{\partial x^2} + \frac{\partial^2 u}{\partial y^2} + \frac{\partial^2 u}{\partial z^2}\right)$$

$$\frac{\partial v}{\partial t} + u \frac{\partial v}{\partial x} + v \frac{\partial v}{\partial y} + w \frac{\partial v}{\partial z} = -\frac{1}{\rho} \frac{\partial p}{\partial y} + \nu \left(\frac{\partial^2 v}{\partial x^2} + \frac{\partial^2 v}{\partial y^2} + \frac{\partial^2 v}{\partial z^2}\right)$$

$$\frac{\partial w}{\partial t} + u \frac{\partial w}{\partial x} + v \frac{\partial w}{\partial y} + w \frac{\partial w}{\partial z} = -\frac{1}{\rho} \frac{\partial p}{\partial z} + \nu \left(\frac{\partial^2 w}{\partial x^2} + \frac{\partial^2 w}{\partial y^2} + \frac{\partial^2 w}{\partial z^2}\right)$$


$$\frac{\partial p}{\partial x} + \frac{\partial p}{\partial y} + \frac{\partial p}{\partial y} = 0$$

where:

* $u$ is the velocity in the $x$-direction
* $v$ is the velocity in the $y$-direction
* $w$ is the velocity in the $z$-direction
* $p$ is the pressure
* $\rho$ is the density
* $\nu$ is the kinematic viscosity

For the first (1.) case, i.e. one dimensional flow through a pipe, the NS equation is reduced to:
$$ \frac{\partial u}{\partial t} + u \frac{\partial u}{\partial x} + v \frac{\partial u}{\partial y} + w \frac{\partial u}{\partial z} = -\frac{1}{\rho} \frac{\partial p}{\partial x} + \nu \left(\frac{\partial^2 u}{\partial x^2} + \frac{\partial^2 u}{\partial y^2} + \frac{\partial^2 u}{\partial z^2}\right)$$
$$\frac{\partial u}{\partial t} = 0$$

The dimensionless first cell height, $y_1^+$, is defined as
$$y_1^+ = \frac{\rho U_\tau y_1}{\mu},$$
where $\rho$ is the fluid density, $U_\tau$ is the friction velocity, $y_1$ is the first cell height, and $\mu$ is the fluid dynamic viscosity.

In [ ]:
cellNumberHeight = 100 # number of cells through the height of the domain
cellWidth = 1.0 # unit width of cells
dlfirstCellHeight = #
